Best score: 0.74102

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import tensorflow as tf

import tflearn
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

submission = pd.read_csv("../input/sample_submission.csv")
submission["type"] = "Unknown"

In [ ]:
#sns.pairplot(train.drop("id",axis=1), hue="type", diag_kind="kde")

### There is a problem with the Goblins wich just are on top of the Ghouls.

# Feature engineering 

### First just manually visualize engineered features.

In [ ]:
import itertools

In [ ]:
comb = list(itertools.combinations(train.drop(["id", "color", "type"], axis=1).columns, 2))

In [ ]:
try_comb = pd.DataFrame()
for c in comb:
    try_comb[c[0]+"_x_"+c[1]] = train[c[0]].values *train[c[1]].values

try_comb["type"] = train.type

In [ ]:
#sns.pairplot(try_comb, hue="type", diag_kind="kde")

In [ ]:
try_comb = None

Keep:

- bone_length_x_hair_length: comb[1]
- bone_length_x_has_soul: comb[2]
- hair_length_x_has_soul: comb[-1]

In [ ]:
for i in [1,2,-1]:
    train[comb[i][0]+"_x_"+comb[i][1]] = train[comb[i][0]].values * train[comb[i][1]].values
    test[comb[i][0]+"_x_"+comb[i][1]] = test[comb[i][0]].values * test[comb[i][1]].values

### Check the projection in 2D using KernelPCA.

In [ ]:
from sklearn.decomposition import KernelPCA

In [ ]:
kPCA = KernelPCA(n_components=2, kernel="rbf", gamma=1)
transf = kPCA.fit_transform(train.drop(["id", "color", "type"], axis=1))

plt.figure(figsize=(10,8))

for label,marker,color in zip(["Ghost", "Ghoul", "Goblin"],('x', 'o', '^'),('blue', 'red', 'green')):

    plt.scatter(x=transf[:,0][(train.type == label).values],
                y=transf[:,1][(train.type == label).values],
                marker=marker,
                color=color,
                alpha=0.7,
                label='class {}'.format(label)
                )

plt.legend()
plt.title('KernelPCA projection')

plt.show()

### Use the projection as new features.

In [ ]:
train["kPCA_0"] = transf[:,0]
train["kPCA_1"] = transf[:,1]

In [ ]:
transf_test = kPCA.transform(test.drop(["id", "color"], axis=1).values)

test["kPCA_0"] = transf_test[:,0]
test["kPCA_1"] = transf_test[:,1]

In [ ]:
#sns.pairplot(train.drop(["id", "color"],axis=1), hue="type", diag_kind="kde")

# Neural Net

In [ ]:
X_train = train.drop(["id", "color", "type"], axis=1)
y_train = pd.get_dummies(train["type"]).values

X_test = test.drop(["id", "color"], axis=1)

In [ ]:
run = True

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [ ]:
scores = []
predictions = np.zeros((X_test.values.shape[0], 3))
for train_idx, val_idx in kf.split(X_train):
    with tf.Graph().as_default():

        net = tflearn.input_data(shape=[None, 9])

        net = tflearn.fully_connected(net, 1024,
                                      activation='relu',
                                      weights_init='xavier',
                                      regularizer='L2')
        net = tflearn.dropout(net, 0.5)

        net = tflearn.fully_connected(net, 3, activation='softmax')

        net = tflearn.regression(net)

        model = tflearn.DNN(net, tensorboard_verbose=0)

        model.fit(X_train.values[train_idx],
                  y_train[train_idx],
                  n_epoch=150)

        score = model.evaluate(X_train.values[val_idx], y_train[val_idx])

        scores.append(score[0])

        print("\n", "SCORE:", score[0], "\n\n")

        prediction = np.array(model.predict(X_test))

        predictions += prediction * score[0]

In [ ]:
scores

In [ ]:
test_pred = np.argmax(predictions, axis=1).astype(str)
test_pred[test_pred=="0"] = "Ghost"
test_pred[test_pred=="1"] = "Ghoul"
test_pred[test_pred=="2"] = "Goblin"
test_pred

In [ ]:
submission["type"] = test_pred

In [ ]:
submission.to_csv("NN.csv", index=False)